# Generate Policies

> The Policy takes a zipped file containing Standard's JSON (2-5) as input. The API splits the controls from the standard and applies an embedding process. It then performs Cosine Similarity on the controls' vector and merges the controls' content, ultimately generating a new set of policies as output. 

In [ ]:
#| default_exp policy/generate_policies

In [ ]:
#| export
from app.policy.helper.llm_functions import get_openai_embedding, init_azure_llm
from app.policy.helper.tools import apply_markdown, apply_markdown_gap, merge_sets, create_policy_markdown
from app.policy.helper.prompts import policy_prompt_template, controls_prompt_template, policy_gap_prompt_template, policy_gap_output
from app.policy.generate_controls import standard_processor
import json
import concurrent.futures


In [ ]:
#| export
# Function to extract headings using LangChain
def extract_headings_using_langchain(
    llm, # LLM Object
    Domain, # `Domain` Object
    domain_name, #  Domain name
    region,industry
):
    prompt = policy_prompt_template.format(current_input=Domain, domain_name=domain_name, region=region,industry=industry)
    # json_llm = llm.bind(response_format={"type": "json_object"})
    response = llm.invoke(prompt)
    return response

In [ ]:
#| export
def policy_standard_processor(
    input_standards:list, # List of standards
    region:str,
    industry:str
)->list: # list of policies of similar domains
    "Generate policies for the similar domains"
    print('policy api process started')
    llm = init_azure_llm()
    standards = standard_processor(input_standards)
    updated_policies = []
    for standard in standards:
        for control in standard['Controls']:
            # print("len(control['id'].split('-'))",len(control['id'].split('-')))
            if len(control['id'].split('-'))>1:
                updated_policies.append(standard)
                break
    # print("updated_policies_count=",len(updated_policies))
    def process_policy(policy):
        m_Domain_Controls = create_policy_markdown(policy)
        response = extract_headings_using_langchain(llm, m_Domain_Controls, policy['domain'],region,industry)
        response_json = (response.content)
        return response_json

    with concurrent.futures.ThreadPoolExecutor() as executor:
        response_json_list = list(executor.map(process_policy, updated_policies))
    print('policy api process completed')
    return response_json_list  #list

In [ ]:
#| code-fold: true
#| eval: false
standard1 = {
    "StandardName": "ISO/IEC 27001:2022",
    "Controls": [
        {
            "DomainId": "4",
            "Domain": "Context of the organization",
            "Controls": [
                {
                    "controlId": "4.1",
                    "policy": "Context of the organization",
                    "name": "Understanding the organization and its context",
                    "description": "The organization shall determine external and internal issues that are relevant to its purpose and that affect its ability to achieve the intended outcome(s) of its information security management system.",
                },
                {
                    "controlId": "4.2",
                    "policy": "Context of the organization",
                    "name": "Understanding the needs and expectations of interested parties",
                    "description": "The organization shall determine: a) interested parties that are relevant to the information security management system; b) the relevant requirements of these interested parties; c) which of these requirements will be addressed through the information security management system.",
                },
            ],
        }
    ],
}
standard2 = {
    "StandardName": "ISO 9001:2015",
    "Controls": [
        {
            "DomainId": "4",
            "Domain": "Context of the organization",
            "Controls": [
                {
                    "controlId": "4.1",
                    "policy": "Context of the organization",
                    "name": "Understanding the organization and its context",
                    "description": "The organization shall determine external and internal issues that are relevant to its purpose and its strategic direction and that affect its ability to achieve the intended result(s) of its quality management system. \nThe organization shall monitor and review information about these external and internal issues. \nNOTE 1 Issues can include positive and negative factors or conditions for consideration.\nNOTE 2 Understanding the external context can be facilitated by considering issues arising from legal, technological, competitive, market, cultural, social and economic environments, whether international, national, regional or local. \nNOTE 3 Understanding the internal context can be facilitated by considering issues related to values, culture, knowledge and performance of the organization.",
                },
                {
                    "controlId": "4.2",
                    "policy": "Context of the organization",
                    "name": "Understanding the needs and expectations of interested parties",
                    "description": "Due to their effect or potential effect on the organization’s ability to consistently provide products and\nservices that meet customer and applicable statutory and regulatory requirements, the organization\nshall determine:\na) the interested parties that are relevant to the quality management system;\nb) the requirements of these interested parties that are relevant to the quality management system.\nThe organization shall monitor and review information about these interested parties and their\nrelevant requirements.",
                },
            ],
        }
    ],
}
policy_standard_processor([standard1,standard2])

policy api process started
control api process started
control api process completed
policy api process completed


[{'Policy Document': [{'Section': 'Introduction',
    'Item': 'Purpose of the Policy',
    'Description': 'The purpose of this policy is to ensure that the organization comprehensively understands its internal and external context to align its information security and quality management systems with strategic objectives. This policy guides employees in adhering to these controls to meet regulatory requirements and achieve intended outcomes.'},
   {'Section': 'Introduction',
    'Item': 'Scope and Applicability',
    'Description': "This policy applies to all employees, departments, and activities governed by the organization's information security and quality management systems. It is mandatory for internal teams and serves as a compliance demonstration tool for external auditors and regulators."},
   {'Section': 'Policy Information',
    'Item': 'Policy Name',
    'Description': 'Context of the organization-Context of the organization'},
   {'Section': 'Policy Information',
    'Item'

In [2]:
#| hide
import nbdev; nbdev.nbdev_export()